In [1]:
#change the kernal to spylon-kernel and then run
#what this pipeline is doing is-this spark code is doing is-> we have list of users who have bunch of users
#but we also have bunch of devices
#users can have many devices but device can also have many users- many to many both sides
#so we are doing here is create both sides users - and all there devices and also a  device - and all there users doing that 
#so one of the thing we do initially is we also want the we doing that with eventsAggregated cfeate it and use it 
#we dont want to do at event level , we wanna aggregate to the userid and device id level 1st in eventsAggregated and go from there
#then take eventsAggregated and join in back to itself and then kinda aggregate up in userAndDevices and in devicesOnEvents
#This pipeline is trivial because we are reusing eventsAggregated dataframe in this pipeline

Intitializing Scala interpreter ...

Spark Web UI available at http://b79bc3db1652:4040
SparkContext available as 'sc' (version = 3.5.5, master = local[*], app id = local-1753464973647)
SparkSession available as 'spark'


<console>: 2: error: ';' expected but '#' found.

In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.storage.StorageLevel

Intitializing Scala interpreter ...

Spark Web UI available at http://b79bc3db1652:4040
SparkContext available as 'sc' (version = 3.5.5, master = local[*], app id = local-1753641519536)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._
import org.apache.spark.storage.StorageLevel


In [2]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")
spark.conf.set("spark.sql.shuffle.partitions", "4")

In [ ]:
# Broadcast join threshold: Disables auto-broadcasting joins.
# Fail ambiguous self join: Allows self joins without errors.
# Shuffle partitions: Limits the number of shuffle partitions (for joins, groupBy, etc.) to 4 for better performance on small data.

In [ ]:
#two types of caching
# .cache() or .persist(StorageLevel.MEMORY_ONLY) which will store data inmemory and can resue it in spark 
# .persist(storageLevel.Disk_ONLY) this is not much usefullunless you createtableand save this to that table likebelow
#eventsAggregated.write.mode("overwrite").saveAsTable("bootcamp.events_aggregated_staging")

In [3]:
val users = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/events.csv")
                        .where($"user_id".isNotNull)

users.createOrReplaceTempView("events") 
# In PySpark: Just replace val with regular variables and drop the $ for column names.

users: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: int, device_id: int ... 4 more fields]


In [ ]:
#users.createOrReplaceTempView("events") -->users dataframe into sql temp table "events" can use events in sql query only for current session
#// Using SQL to query your DataFrame
# val highActivity = spark.sql("""
#   SELECT user_id, COUNT(*) as cnt
#   FROM events
#   GROUP BY user_id
#   HAVING cnt > 10
# """)

In [4]:
users.show(5)

+-----------+---------+--------+--------------------+---+--------------------+
|    user_id|device_id|referrer|                host|url|          event_time|
+-----------+---------+--------+--------------------+---+--------------------+
| 1037710827|532630305|    NULL| www.zachwilson.tech|  /|2021-03-08 17:27:...|
|  925588856|532630305|    NULL|    www.eczachly.com|  /|2021-05-10 11:26:...|
|-1180485268|532630305|    NULL|admin.zachwilson....|  /|2021-02-17 16:19:...|
|-1044833855|532630305|    NULL| www.zachwilson.tech|  /|2021-09-24 15:53:...|
|  747494706|532630305|    NULL| www.zachwilson.tech|  /|2021-09-26 16:03:...|
+-----------+---------+--------+--------------------+---+--------------------+
only showing top 5 rows



In [5]:
spark.sql("SELECT * FROM events LIMIT 10").show()

+-----------+---------+--------+--------------------+---+--------------------+
|    user_id|device_id|referrer|                host|url|          event_time|
+-----------+---------+--------+--------------------+---+--------------------+
| 1037710827|532630305|    NULL| www.zachwilson.tech|  /|2021-03-08 17:27:...|
|  925588856|532630305|    NULL|    www.eczachly.com|  /|2021-05-10 11:26:...|
|-1180485268|532630305|    NULL|admin.zachwilson....|  /|2021-02-17 16:19:...|
|-1044833855|532630305|    NULL| www.zachwilson.tech|  /|2021-09-24 15:53:...|
|  747494706|532630305|    NULL| www.zachwilson.tech|  /|2021-09-26 16:03:...|
|  747494706|532630305|    NULL|admin.zachwilson....|  /|2021-02-21 16:08:...|
| -824540328|532630305|    NULL|admin.zachwilson....|  /|2021-09-28 17:23:...|
| -824540328|532630305|    NULL|    www.eczachly.com|  /|2021-09-29 01:22:...|
| 1833036683|532630305|    NULL|admin.zachwilson....|  /|2021-01-24 03:15:...|
|-2134824313|532630305|    NULL|    www.eczachly.com

In [9]:
 # Using SQL to query your DataFrame
val highActivity = spark.sql("""
  SELECT user_id, COUNT(*) as cnt
  FROM events
  GROUP BY user_id
  HAVING cnt > 10
""")
# This query is running on your DataFrame, not an actual database table! You have created a DataFrame called highActivity.

highActivity: org.apache.spark.sql.DataFrame = [user_id: int, cnt: bigint]


In [10]:
highActivity.show(10)

+-----------+---+
|    user_id|cnt|
+-----------+---+
| -173749422| 16|
| 1399665425| 16|
|  632739597| 16|
| 1072106763| 33|
|  210988258| 32|
|   18650769| 25|
|-1072136053| 20|
|  203689086| 21|
| 1231244260| 13|
|-1563553498| 24|
+-----------+---+
only showing top 10 rows



In [6]:
users.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- referrer: string (nullable = true)
 |-- host: string (nullable = true)
 |-- url: string (nullable = true)
 |-- event_time: timestamp (nullable = true)



In [7]:
val devices = spark.read.option("header", "true")
                       .option("inferSchema", "true")
                       .csv("/home/iceberg/data/devices.csv")

devices.createOrReplaceTempView("devices")


devices: org.apache.spark.sql.DataFrame = [device_id: int, browser_type: string ... 2 more fields]


In [8]:
devices.show(5)

+-----------+--------------------+-------+------------------+
|  device_id|        browser_type|os_type|       device_type|
+-----------+--------------------+-------+------------------+
|-2147042689|             Firefox| Ubuntu|             Other|
|-2146219609|            WhatsApp|  Other|            Spider|
|-2145574618|       Chrome Mobile|Android|Generic Smartphone|
|-2144707350|Chrome Mobile Web...|Android|  Samsung SM-G988B|
|-2143813999|Mobile Safari UI/...|    iOS|            iPhone|
+-----------+--------------------+-------+------------------+
only showing top 5 rows



In [9]:
devices.printSchema()

root
 |-- device_id: integer (nullable = true)
 |-- browser_type: string (nullable = true)
 |-- os_type: string (nullable = true)
 |-- device_type: string (nullable = true)



In [10]:
val eventsAggregated = spark.sql(f"""
  SELECT user_id, 
         device_id, 
         COUNT(1) as event_counts, 
         COLLECT_LIST(DISTINCT host) as host_array
  FROM events
  GROUP BY 1,2
""").cache()

eventsAggregated: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: int, device_id: int ... 2 more fields]


In [11]:
eventsAggregated.show(10)
eventsAggregated.printSchema()
eventsAggregated.count()

+-----------+-----------+------------+--------------------+
|    user_id|  device_id|event_counts|          host_array|
+-----------+-----------+------------+--------------------+
|-2147421007| -807271869|           1|[admin.zachwilson...|
|-2147340867| 1324700293|           1|  [www.eczachly.com]|
|-2147051672|  583904608|           1|  [www.eczachly.com]|
|-2146961540| 1436415199|           1|   [dataengineer.io]|
|-2146752203| -887516106|           1|  [www.eczachly.com]|
|-2146501484| -240714045|           1|  [www.eczachly.com]|
|-2146204366|-1893334816|           1|  [www.eczachly.com]|
|-2146151971| 1177446421|           5|  [www.eczachly.com]|
|-2146090945|-1238295066|           1|  [www.eczachly.com]|
|-2146031264| 1439772194|           1|  [www.eczachly.com]|
+-----------+-----------+------------+--------------------+
only showing top 10 rows

root
 |-- user_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- event_counts: long (nullable = false)
 |--

res8: Long = 141935


In [12]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bootcamp.events_aggregated_staging (
        user_id BIGINT,
        device_id BIGINT,
        event_counts BIGINT,
        host_array ARRAY<STRING>
    )
    PARTITIONED BY (ds STRING)
""")

# This just creates a table definition in your metastore/catalog.
# Data is NOT written yet (your save command is commented out).

res9: org.apache.spark.sql.DataFrame = []


In [14]:
val usersAndDevices = users
  .join(eventsAggregated, eventsAggregated("user_id") === users("user_id"))
  .groupBy(users("user_id"))
  .agg(
    users("user_id"),
    max(eventsAggregated("event_counts")).as("total_hits"),
    collect_list(eventsAggregated("device_id")).as("devices")
  )


usersAndDevices: org.apache.spark.sql.DataFrame = [user_id: int, user_id: int ... 2 more fields]


In [ ]:
# Joins users and aggregated events by user_id.
# Groups by user, finds max event count (per user), and collects device_ids per user.

In [15]:
usersAndDevices.show(10, truncate=false)
usersAndDevices.printSchema()

+-----------+-----------+----------+----------------------------------------------+
|user_id    |user_id    |total_hits|devices                                       |
+-----------+-----------+----------+----------------------------------------------+
|-2147470439|-2147470439|3         |[378988111, 378988111, 378988111]             |
|-2147421007|-2147421007|1         |[-807271869]                                  |
|-2147326548|-2147326548|1         |[532630305]                                   |
|-2147051672|-2147051672|1         |[1237528510, 1237528510, 583904608, 583904608]|
|-2146931906|-2146931906|1         |[532630305]                                   |
|-2146892655|-2146892655|2         |[-1516742733, -1516742733]                    |
|-2146848244|-2146848244|1         |[891045274]                                   |
|-2146743833|-2146743833|1         |[-258252617]                                  |
|-2146598128|-2146598128|1         |[-723516686]                            

In [16]:
val devicesOnEvents = devices
  .join(eventsAggregated, devices("device_id") === eventsAggregated("device_id"))
  .groupBy(devices("device_id"), devices("device_type"))
  .agg(
    devices("device_id"),
    devices("device_type"),
    collect_list(eventsAggregated("user_id")).as("users")
  )

devicesOnEvents: org.apache.spark.sql.DataFrame = [device_id: int, device_type: string ... 3 more fields]


In [ ]:
# Joins devices with aggregated events on device_id.
# Groups by device & type, collects all user_ids who used the device.

In [17]:
devicesOnEvents.show(10, truncate=false)
devicesOnEvents.printSchema()

+-----------+----------------+-----------+----------------+---------------------------------------------------+
|device_id  |device_type     |device_id  |device_type     |users                                              |
+-----------+----------------+-----------+----------------+---------------------------------------------------+
|-2147042689|Other           |-2147042689|Other           |[1633522354]                                       |
|-2146219609|Spider          |-2146219609|Spider          |[-2119549067]                                      |
|-2144707350|Samsung SM-G988B|-2144707350|Samsung SM-G988B|[332947552]                                        |
|-2143813999|iPhone          |-2143813999|iPhone          |[-945726225]                                       |
|-2142634982|iPhone          |-2142634982|iPhone          |[47777302]                                         |
|-2142350383|iPhone          |-2142350383|iPhone          |[-1744927308, 1798466603, -1431608087, -13954

In [3]:
import org.apache.spark.sql.functions._
import org.apache.spark.storage.StorageLevel

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")
spark.conf.set("spark.sql.shuffle.partitions", "4")

val users = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/events.csv")
                        .where($"user_id".isNotNull)

users.createOrReplaceTempView("events")

val devices = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/devices.csv")

devices.createOrReplaceTempView("devices")

val executionDate = "2023-01-01"

# //Caching here should be < 5 GBs or used for broadcast join
# //You need to tune executor memory otherwise it'll spill to disk and be slow
# //Donreally try using any of the other StorageLevel besides MEMORY_ONLY

val eventsAggregated = spark.sql(f"""
  SELECT user_id, 
          device_id, 
        COUNT(1) as event_counts, 
        COLLECT_LIST(DISTINCT host) as host_array
  FROM events
  GROUP BY 1,2
""").cache()

// eventsAggregated.write.mode("overwrite").saveAsTable("bootcamp.events_aggregated_staging")

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bootcamp.events_aggregated_staging (
        user_id BIGINT,
        device_id BIGINT,
        event_counts BIGINT,
        host_array ARRAY<STRING>
    )
    PARTITIONED BY (ds STRING)
""")


val usersAndDevices = users
  .join(eventsAggregated, eventsAggregated("user_id") === users("user_id"))
  .groupBy(users("user_id"))
  .agg(
    users("user_id"),
    max(eventsAggregated("event_counts")).as("total_hits"),
    collect_list(eventsAggregated("device_id")).as("devices")
  )

val devicesOnEvents = devices
      .join(eventsAggregated, devices("device_id") === eventsAggregated("device_id"))
      .groupBy(devices("device_id"), devices("device_type"))
      .agg(
        devices("device_id"),
        devices("device_type"),
         collect_list(eventsAggregated("user_id")).as("users")
      )

devicesOnEvents.explain()
usersAndDevices.explain()

devicesOnEvents.take(1)
usersAndDevices.take(1)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- ObjectHashAggregate(keys=[device_id#598, device_type#601], functions=[collect_list(user_id#568, 0, 0)])
   +- ObjectHashAggregate(keys=[device_id#598, device_type#601], functions=[partial_collect_list(user_id#568, 0, 0)])
      +- Project [device_id#598, device_type#601, user_id#568]
         +- SortMergeJoin [device_id#598], [device_id#569], Inner
            :- Sort [device_id#598 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(device_id#598, 4), ENSURE_REQUIREMENTS, [plan_id=735]
            :     +- Filter isnotnull(device_id#598)
            :        +- FileScan csv [device_id#598,device_type#601] Batched: false, DataFilters: [isnotnull(device_id#598)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/devices.csv], PartitionFilters: [], PushedFilters: [IsNotNull(device_id)], ReadSchema: struct<device_id:int,device_type:string>
            +- Sort [device_id#569 ASC NULLS FIRS

import org.apache.spark.sql.functions._
import org.apache.spark.storage.StorageLevel
users: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: int, device_id: int ... 4 more fields]
devices: org.apache.spark.sql.DataFrame = [device_id: int, browser_type: string ... 2 more fields]
executionDate: String = 2023-01-01
eventsAggregated: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: int, device_id: int ... 2 more fields]
usersAndDevices: org.apache.spark.sql.DataFrame = [user_id: int, user_id: int ... 2 more fields]
devicesOnEvents: org.apache.spark.sql.DataFrame = [device_id: int, device_type: string ... 3 more fields]
res1: Array[org.apache.spark.sql.Row] = Array([-2147470439,-2147470439,3,WrappedArray(378988111, 378988111, 378988111)])


In [19]:
eventsAggregated.unpersist()

res18: eventsAggregated.type = [user_id: int, device_id: int ... 2 more fields]
